In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mplt
import numpy as np
import regex as re

In [ ]:
df = pd.read_csv('properatti.csv')

In [ ]:
df.head()
df.columns
df.rename(columns={"Unnamed: 0":"number"}, inplace=True)

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
df.info()

In [ ]:
df.columns[df.isna().any()].tolist()

In [ ]:
df["currency"].value_counts()

In [ ]:
subset= df[["price","currency","price_aprox_local_currency","price_aprox_usd"]]

In [ ]:
subset["price"].isna().sum()

In [ ]:
subset["currency"].isna().sum()

In [ ]:
subset["price_aprox_local_currency"].isna().sum()

In [ ]:
subset["price_aprox_usd"].isna().sum()

In [ ]:
subset[subset.currency.isna()]

In [ ]:
df[df.currency.isna()].head(50)

In [ ]:
df[df.price.isna()][["price","currency","price_aprox_local_currency","price_aprox_usd","price_usd_per_m2","surface_covered_in_m2","surface_total_in_m2","price_per_m2"]].head(50)

In [ ]:
df[df.currency.isna()]["surface_covered_in_m2"].isna().sum()

In [ ]:
#df["state_name"].value_counts()

In [ ]:
#Acá se obtienen los precios de la columna "description" con filtros de regex.
#Los resultados se guardan en el diccionario "dict_prices_found" con la etiqueta de la fila a la 
#que corresponde, y dentro de ésta una lista con todos los precios encontrados, donde se etiquetan por
#monto y divisa.

currency_filters = ["u[$s][ds$]\s\d+\.\d\d\d\.\d\d\d","u[$s][ds$]\d+\.\d\d\d\.\d\d\d","u[$s][ds$]\s\d+\.\d\d\d","u[$s][ds$]\d+\.\d\d\d","u[$s][ds$]\s\d+","u[$s][ds$]\d+","ar[s$]\s\d+\.\d\d\d\.\d\d\d","ar[s$]\d+\.\d\d\d\.\d\d\d","ar[s$]\s\d+\.\d\d\d","ar[s$]\d+\.\d\d\d","ar[s$]\s\d+","ar[s$]\d+","uyu\s\d+\.\d\d\d\.\d\d\d","uyu\d+\.\d\d\d\.\d\d\d","uyu\s\d+\.\d\d\d","uyu\d+\.\d\d\d","uyu\s\d+","uyu\d+","pen\s\d+\.\d\d\d\.\d\d\d","pen\d+\.\d\d\d\.\d\d\d","pen\s\d+\.\d\d\d","pen\d+\.\d\d\d","pen\s\d+","pen\d+"]
regexResult = []

dict_prices_found = {}
for row in df['number']:
    
    description = df['description'][row]
    currency_found = False
    for x in range(len(currency_filters)):
        currency = currency_filters[x]
        
        regexMatch = re.compile(currency, flags = re.IGNORECASE).findall(str(description))
                
        if len(regexMatch) > 0:     #Si es un dato válido...
            actual_currency='no_currency' #Inicializa en un valor sin moneda.
            possible_prices = []    # Inicializo lista vacía por cada búsqueda.
            #possible_prices = [str(row)]    # PRUEBA PARA DEBUGGEAR!!
            currency_found = True   
    
            for i in regexMatch:
                if x in range(0,6):   #Es USD
                    actual_currency = "USD"
                elif x in range(6,12):   #Es ARS
                    actual_currency = "ARS"
                elif x in range(12,18):  #Es UYU
                    actual_currency = "UYU"
                elif x in range(18,24): #Es PEN
                    actual_currency = "PEN"   
                
                amount = re.compile("\d+\.\d+").findall(str(i))  #Se buscan solo cifras numéricas.
                if len(amount) == 0:   #Si no hubo coincidencias...
                    amount = re.compile("\d+").findall(str(i))   #Se buscan conjuntos de una sola cifra.
                  
                amount = float(amount[0].replace(".",""))   #Sacamos el punto decimal.  
                #print(row, i, amount )
                possible_prices.append( {"amount" : amount, "currency" : actual_currency} )  #Se agrega a los posibles precios.
                 
            dict_prices_found[str(row)] = possible_prices
            break
    if currency_found == False:
        pass #Se agrega None para mantener orden de las filas
            
#regexResult
dict_prices_found

In [ ]:
#De todas las filas que tengan precio y superficie válidas, los obtiene y los guarda en una lista,
#la cual está catalogada por zona/barrio, tipo de propiedad y divisa.

#dict_prices={ "place_name": {"property_type":{"currency 1":[1,2,3,4,5,6], "currency 2":[1,2,3,4,5,6]}} }
dict_prices = {}

for row in df['number']:        # Para cada fila de la lista...
    property_type = df["property_type"][row]        # Tipo de propiedad
    place_name = df["place_name"][row]              # Barrio
    price = df["price"][row]
    currency = df["currency"][row]
    total_surface = df["surface_total_in_m2"][row]  # Superficie
                    
    #Si hay una superficie y precio válidos...
    if type(currency) is str:                             
        if type(total_surface) is not np.nan and type(price) is not np.nan:  
             if total_surface > 0 and price > 0:
        
                price_per_m2 = price / total_surface    # Precio por m^2

                if place_name not in dict_prices.keys():  # Si no existe la localidad...
                    dict_prices[place_name] = {}   #La inicaliza con una lista con un diccionario dentro
                
                if property_type not in dict_prices[place_name].keys():
                    dict_prices[place_name][property_type] = {}
                
                if currency not in dict_prices[place_name][property_type].keys():    #Si no existe la divisa
                    dict_prices[place_name][property_type][currency]=[]    #La inicializa con una lista vacía donde irán los valores.

                dict_prices[place_name][property_type][currency].append( price_per_m2 )
#############################     
dict_prices

In [ ]:
#Partiendo de los precios obtenidos en el diccionario "dict_prices", toma todos los valores y obtiene el mínimo 
#y máximo por m^2, catalogado por tipo de propiedad (property_type) y zona/barrio (place_name).
#a su vez, todos están separados por divisa. La jerarquìa de este diccionario es la siguiente:
#dict_price_by_place = {'place_name':{"property_type":{'currency':{'minimum':[1,2,3,4], 'maximum':[1,2,3,4]}}} }  

dict_price_by_place = {}
for place_name in dict_prices.keys():
    #print('\n-----Localidad: {loc}'.format(loc=place_name))

    for property_type in dict_prices[place_name].keys():
        #print('\nTipo de propiedad: {prop}'.format(prop=property_type))
        
        for currency in dict_prices[place_name][property_type].keys():

            if place_name not in dict_price_by_place.keys():
                dict_price_by_place[place_name]={}
            
            if property_type not in dict_price_by_place[place_name].keys():
                dict_price_by_place[place_name][property_type]={}
            
            if currency not in dict_price_by_place[place_name][property_type]:
                    dict_price_by_place[place_name][property_type][currency]={}
                    dict_price_by_place[place_name][property_type][currency]['maximum']=[]
                    dict_price_by_place[place_name][property_type][currency]['minimum']=[]
            #Guarda precio máximo y mínimo.
            dict_price_by_place[place_name][property_type][currency]['maximum'].append(max(dict_prices[place_name][property_type][currency]))
            dict_price_by_place[place_name][property_type][currency]['minimum'].append(min(dict_prices[place_name][property_type][currency]))

dict_price_by_place

In [ ]:
#Parte del diccionario de precios obtenidos "dict_prices_found", solo analizando las filas donde los hay. 
#si no tienen un precio válido y si tienen una superficie válida, continúa:
# Partiendo de la información de referencia obtenida en el diccionario "dict_price_by_place", verifica
# que la fila actual contenga información válida que se pueda comparar con ésta, a modo de filtrar los valores 
# obtenidos que no sean "coherentes".
# Si los hay, calcula el precio por m^2 de la propiedad actual y lo compara con la referencia, teniendo
# en cuenta el valor asociado a la zona/barrio y el tipo de propiedad, sea en la divisa que sea.
# Si el valor está dentro del mínimo y el máximo, se considera como precio válido y se guarda en el diccionario 
# "dict_final_prices". 
# En el caso de que una propiedad tuviera más de un posible precio final (porque ambos pasaron la condición
# de ser "coherentes"), estos precios se guardan en el diccionario "dict_multiple_prices", con el cual se podría
# tomar estos pocos casos y analizarlos manualmente.
# El formato de estos diccionarios es: 
#
# {'row': {'amount': 115000.0, 'currency': 'USD'} } #Por cada fila hay un monto y una divisa.
# En el caso del "dict_multiple_prices", dentro de la etiqueta "row" hay una lista compuesta por diccionarios
# similares.

dict_final_prices = {}     #Lista de salida con los precios averiguados.
dict_multiple_prices = {}

for row in dict_prices_found.keys():
    #print("row:"+str(row))
    
    prices_found = dict_prices_found[row] #Lista de precios encontrados en la descripción.
    
    row=int(row)    #Casetamos a int para usar el número y seleccionar la fila en cuestión
    
    #Obtenemos los datos importantes para cada fila: Barrio, tipo de propiedad, precio y superficie.
    place_name = df["place_name"][row]
    property_type = df["property_type"][row]
    price_total = df["price"][row]
    total_surface = df["surface_total_in_m2"][row]
        
    #print(row,place_name +':'+ property_type, price_total, dict_prices_found[str(row)] , total_surface)
    
    if not price_total > 0 and total_surface > 0:  #Si no hay precio en la columna "price" y la superficie es > 0
        
        if place_name is not np.nan and property_type is not np.nan:   #Si el barrio y tipo de propiedad están especificados...
            #print(place_name, property_type)

            if place_name in dict_price_by_place.keys():   #Si tenemos referencia del barrio...
                #print(place_name, property_type)
                if property_type in dict_price_by_place[place_name]:   #Si tenemos referencia del tipo de propiedad...
                    #Pasamos a analizar los posibles precios:
                    
                    prices_found = dict_prices_found[str(row)]    #Precios encontrados en la descripción de esta propiedad.
    
                    for actual_price in prices_found:    #Por cada uno de los precios encontrados para esta propiedad...
                        #print("row:"+str(row)+" precio encontrado:"+str(actual_price))
                        actual_amount = actual_price["amount"]
                        actual_currency = actual_price["currency"]
                    
                        valid_prices = []
                        
                        #Si la divisa existe en la referencia...
                        if actual_currency in dict_price_by_place[place_name][property_type].keys():
                            price_max = dict_price_by_place[place_name][property_type][actual_currency]["maximum"]
                            price_min = dict_price_by_place[place_name][property_type][actual_currency]["minimum"]
                            
                            #Calcular precio por m2
                            price_per_m2 = actual_amount / total_surface
                            
                            #Compararla con el rango minimo y máximo.
                            if price_per_m2 >= price_min and price_per_m2 <= price_max: 
                                #Si este precio es válido, se carga en una lista de precios válidos (valid_prices)
                                
                                #Guardamos el precio encontrado como válido
                                valid_prices.append({"amount":actual_amount, "currency":actual_currency})  
                    #print(row,len(valid_prices))

                            
                    
                    #Si los precios válidos son más de uno, con cuál nos quedamos?
                    if len(valid_prices)== 1:
                        dict_final_prices[str(row)] = valid_prices[0]
                    elif len(valid_prices) > 1:   #Si los precios válidos son más de uno...
                        dict_multiple_prices[str(row)] = valid_prices
                    else:      #Si no hay precios válidos..
                        pass
                
# dict_price_by_place[place_name][property_type][currency]{"minimum":[] , "maximum":[]}

dict_final_prices

In [ ]:
#Genera una tabla donde hay que agregar la información obtenida
# formato: [[row, amount, currency, aprox_in_local_currency, aprox_in_usd] ... ]

list_new_values = []

for i in dict_final_prices:
    dict_cotizaciones_usd = {"USD":1 ,"ARS": 17.645, "PEN": 1, "UYU": 1}

    currency = dict_final_prices[i]["currency"]
    amount = dict_final_prices[i]["amount"]
    
    cotizacion_usd = 17.6445
    
    if currency == 'USD':
        aprox_usd = amount
        aprox_local = aprox_usd * cotizacion_usd
    else:    
        aprox_local = amount 
        aprox_usd = amount / cotizacion_usd
        # row                        amount                      currency         local_currency   USD
    values = [int(i), amount, currency, aprox_local, aprox_usd]
    
    list_new_values.append(values)

list_new_values